#### Chain 사용법

```python

chain_one = template | chat | outputparser_one

chain_two = template | chat | outputparser_two

all = chain_one | chain_two | output

```

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [29]:
chef_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
        ("human", "I want to cook {cuisine} food."),
    ]
)

In [30]:
chef_chain = chef_prompt | chat

In [31]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', "Your are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
        ("human", "{recipe}"),
    ]
)

In [32]:
veg_chain = veg_chef_prompt | chat

In [33]:
final_chain = {"recipe" : chef_chain} | veg_chain

In [34]:
final_chain.invoke(
    {
        "cuisine" : "indian",
    }
)

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Chicken Tikka Masala

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon ground turmeric
- 1 teaspoon chili powder
- 1 teaspoon paprika
- 1 teaspoon garam masala
- 2 cloves garlic, minced
- 1-inch piece of ginger, grated
- Salt and pepper to taste
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, combine yogurt, lemon juice, cumin, coriander, turmeric, chili powder, paprika, garam masala, garlic, ginger, salt, and pepper. Add the chicken pieces and mix well to coat. Cover and marinate in the refrigerator for at least 1 hou

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (use plant-based yogurt for a vegan version)\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons ground coriander\n- 1 teaspoon ground turmeric\n- 1 teaspoon chili powder\n- 1 teaspoon paprika\n- 1 teaspoon garam masala\n- 2 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- Salt and pepper to taste\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 1 can (14 oz) tomato sauce\n- 1 cup coconut cream (or plant-based heavy cream)\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but substitute the chicken with tofu or paneer. Make sure to press the tofu to remove excess water before marinating.\n\n2